In [2]:
import pandas as pd
import os
import numpy as np
import datetime
print(datetime.datetime.now())

# Revised based on the 1st version new run, remove the 8+22 together

2019-09-11 13:38:30.510215


In [3]:
os.listdir("./")

['BL_final_TA_updated_JL_2019-07-18.xlsx',
 'output_by_store_zip_sales_cum_60Plus_only_JL_2019-07-26.csv',
 'output_by_store_zip_sales_cum_JL_2019-06-27.csv',
 '.ipynb_checkpoints',
 'remove_8_closing_stores.ipynb',
 'BL_final_TA_updated_remove_8_closing_stores_JL_2019-08-20.xlsx',
 'remove_22_closing_stores_20190911.ipynb',
 'Stores to Remove From BL Footprint 9.4.xlsx']

In [1]:
df_closing_22_stores=pd.read_excel("./Stores to Remove From BL Footprint 9.4.xlsx",dtype=str,sheet_name="Stores to Remove from Footprint")
list_closing_22_stores=df_closing_22_stores['Store ID'].unique().tolist()
len(list_closing_22_stores)

list_closing_8_stores=['182','1095','1499','4061','4070','4125','4441','4529']

total_closing_stores=list(set(list_closing_8_stores+list_closing_22_stores))
len(total_closing_stores)

NameError: name 'pd' is not defined

In [9]:
output_20190718=pd.ExcelFile("./BL_final_TA_updated_JL_2019-07-18.xlsx")
output_20190718.sheet_names

['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [10]:
df_old_by_store=output_20190718.parse("view_by_store",dtype=str)
df_old_by_store['TA_num']=df_old_by_store['TA_num'].astype(int)
df_old_by_store_closing=df_old_by_store[df_old_by_store['location_id'].isin(total_closing_stores)]

In [11]:
df_old_by_store_closing['trans_P_zips_70_within_TA']=df_old_by_store_closing['trans_P_zips_70_within_TA'].apply(eval)
df_old_by_store_closing['trans_S_zips_70_within_TA']=df_old_by_store_closing['trans_S_zips_70_within_TA'].apply(eval)

list_zips_for_closing_stores=df_old_by_store_closing['trans_P_zips_70_within_TA'].sum()+df_old_by_store_closing['trans_S_zips_70_within_TA'].sum()
list_zips_for_closing_stores=list(set(list_zips_for_closing_stores))

df_new_by_store=df_old_by_store[~df_old_by_store['location_id'].isin(total_closing_stores)]
df_new_by_store=df_new_by_store.sort_values("TA_num")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
df_new_by_store['zips_in_10']=df_new_by_store['zips_in_10'].apply(eval)
df_new_by_store['trans_P_zips_70_within_TA']=df_new_by_store['trans_P_zips_70_within_TA'].apply(eval)
df_new_by_store['trans_S_zips_70_within_TA']=df_new_by_store['trans_S_zips_70_within_TA'].apply(eval)

df_new_by_store.shape

(1378, 17)

In [13]:
df_new_by_TA_stores=df_new_by_store.groupby(["TA_num",'ta_name'])['location_id'].apply(list).to_frame().reset_index().rename(columns={"location_id":"store_list"})
df_new_by_TA_P_S_10_zips=df_new_by_store.groupby(["TA_num",'ta_name'])['trans_P_zips_70_within_TA','trans_S_zips_70_within_TA','zips_in_10'].sum().reset_index()
df_new_by_TA_P_S_10_zips=df_new_by_TA_P_S_10_zips.rename(columns={"trans_P_zips_70_within_TA":"trans_P_zips","trans_S_zips_70_within_TA":"trans_S_zips","zips_in_10":"distance_10_zips"})
df_new_by_TA=pd.merge(df_new_by_TA_stores,df_new_by_TA_P_S_10_zips,on=["TA_num",'ta_name'],how="outer")
print(df_new_by_TA.shape)

for col in ['store_list','trans_P_zips','trans_S_zips','distance_10_zips']:
    df_new_by_TA[col]=df_new_by_TA[col].apply(lambda x: list(set(x)))
df_new_by_TA['store_count']=df_new_by_TA['store_list'].apply(len)
df_new_by_TA=df_new_by_TA.sort_values("TA_num")
df_new_by_TA.head(2)

(697, 6)


,TA_num,ta_name,store_list,trans_P_zips,trans_S_zips,distance_10_zips,store_count
0,1,multiple_1,"[1027, 4096, 1860, 1255, 1650, 1179, 1487, 110...","[76111, 75090, 76092, 76201, 75020, 76208, 750...","[75060, 75180, 75253, 75234, 75091, 76118, 762...",[],21
1,2,multiple_2,"[5356, 122, 5373, 1666, 5220, 77, 5369, 1079, ...","[43221, 43212, 43220, 44907, 43078, 43119, 432...","[43326, 43031, 43123, 44813, 43074, 43011, 432...",[],17


In [14]:
df_old_by_TA=output_20190718.parse("view_by_TA",dtype=str)
print(df_old_by_TA.shape)
df_old_by_TA.head(2)

(706, 7)


,TA_num,ta_name,store_list,trans_P_zips,trans_S_zips,distance_10_zips,store_count
0,1,multiple_1,"['1949', '1255', '4543', '1027', '1072', '1106...","['75002', '75006', '75007', '75020', '75023', ...","['73448', '73459', '74701', '74733', '75001', ...",nan,21
1,2,multiple_2,"['5220', '5356', '1666', '1667', '1668', '77',...","['43015', '43016', '43017', '43021', '43026', ...","['43003', '43004', '43008', '43009', '43011', ...",nan,17


In [15]:
df_old_unique_zips_full_footprint=output_20190718.parse("unique_zips_full_footprint",dtype=str)
print(df_old_unique_zips_full_footprint.shape)
df_old_unique_zips_full_footprint['zip_type'].unique()

(12980, 2)


array(['trans_P', 'trans_S', 'zips_10'], dtype=object)

In [16]:
df_new_unique_zips_full_footprint=pd.DataFrame(columns={"zip_type",'zip_cd'})
#
all_P_zips=list(set(df_new_by_TA['trans_P_zips'].sum()))
all_S_zips=list(set(df_new_by_TA['trans_S_zips'].sum()))
all_10_zips=list(set(df_new_by_TA['distance_10_zips'].sum()))
#
all_S_zips=[x for x in all_S_zips if x not in all_P_zips]
#
all_10_zips=[x for x in all_10_zips if x not in all_P_zips]
all_10_zips=[x for x in all_10_zips if x not in all_S_zips]


df_P_zips=pd.DataFrame({"zip_type":["trans_P"]*len(all_P_zips),"zip_cd":all_P_zips},index=[x for x in range(len(all_P_zips))])

df_S_zips=pd.DataFrame({"zip_type":["trans_S"]*len(all_S_zips),"zip_cd":all_S_zips},index=[x for x in range(len(all_S_zips))])

df_10_zips=pd.DataFrame({"zip_type":["zips_10"]*len(all_10_zips),"zip_cd":all_10_zips},index=[x for x in range(len(all_10_zips))])
df_new_unique_zips_full_footprint=pd.concat([df_P_zips,df_S_zips,df_10_zips])
print(df_new_unique_zips_full_footprint.shape)

(12800, 2)


In [17]:
df_new_unique_zips_full_footprint=df_new_unique_zips_full_footprint[['zip_type','zip_cd']]

In [21]:
writer=pd.ExcelWriter("./BL_final_TA_updated_remove_8_and_22_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_new_unique_zips_full_footprint.to_excel(writer,"unique_zips_full_footprint",index=False)
df_new_by_TA.to_excel(writer,"view_by_TA",index=False)
df_new_by_store.to_excel(writer,"view_by_store",index=False)
df_store_removed=pd.DataFrame({"removed_stores":total_closing_stores},index=[x for x in range(len(total_closing_stores))])
df_store_removed.to_excel(writer,"stores_removed",index=False)
writer.save()